## Genre Genie - Multi-label Classification with NLP
### Part 3.2: Fit Best Model using optimized hyperparameters

#### Tom Keith

---

**Goal:** Fit and pickle the best models using all of the data (TF-IDF, standard scaler, OneVsRest with Logistic Regression). Will end up with 3 `my_best_model.pkl` when complete.

Much of the code in this notebook is NOT unique and repurposed from other exploratory notebooks so the commenting is lighter.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
#import scipy.sparse
%matplotlib inline
import csv

In [6]:
df = pd.read_csv('data/clean_df.tsv', sep='\t', index_col=0)
#df.head(10)

In [7]:
cols = list(df.columns.values)

genre_cols = ['g_'+g for g in cols[8:-1]]
print(len(genre_cols), genre_cols)

feature_cols = ['f_'+f for f in cols[:8]]
print(len(feature_cols), feature_cols)

cols = feature_cols + genre_cols + ['plot_long']
df.columns = cols

22 ['g_action', 'g_adventure', 'g_animation', 'g_biography', 'g_comedy', 'g_crime', 'g_documentary', 'g_drama', 'g_family', 'g_fantasy', 'g_film-noir', 'g_history', 'g_horror', 'g_music', 'g_musical', 'g_mystery', 'g_romance', 'g_sci-fi', 'g_sport', 'g_thriller', 'g_war', 'g_western']
8 ['f_release_year', 'f_release_month', 'f_runtime', 'f_word_count_long', 'f_imdb_rating', 'f_num_imdb_votes', 'f_num_user_reviews', 'f_num_critic_reviews']


In [8]:
# X (reviews) and y (target/score)
X = df.drop(genre_cols, axis=1)
y = df[genre_cols]

In [9]:
X.columns

Index(['f_release_year', 'f_release_month', 'f_runtime', 'f_word_count_long',
       'f_imdb_rating', 'f_num_imdb_votes', 'f_num_user_reviews',
       'f_num_critic_reviews', 'plot_long'],
      dtype='object')

---

In [10]:
#Importing the packages we will be using
# Basic Packages
import numpy as np
import pandas as pd

# NLTK Packages
import nltk
# Use the code below to download the NLTK package, a straightforward GUI should pop up
# nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [11]:
from nltk.tokenize import word_tokenize
def tokenizer(text):
    """
    Tokenizes the document
    """
    return word_tokenize(text)


from nltk.corpus import stopwords

#Load up our stop words
stop_words = stopwords.words('english')
#Adds stuff to our stop words list
stop_words.extend(['.',','])


## This function can improve, simplify. Look into Text Data Lecture
def remove_stopwords(list_of_tokens):
    """
    Removes stopwords
    """
    
    cleaned_tokens = [] 
    
    for token in list_of_tokens: 
        if token in stop_words: continue 
        cleaned_tokens.append(token)
            
    return cleaned_tokens


from nltk.stem import PorterStemmer

def stemmer(list_of_tokens):
    '''
    Takes in an input which is a list of tokens, and spits out a list of stemmed tokens.
    '''
    
    stemmed_tokens_list = []
    
    for i in list_of_tokens:
        
        token = PorterStemmer().stem(i)
        stemmed_tokens_list.append(token)
        
    return stemmed_tokens_list


from nltk.stem import WordNetLemmatizer

def lemmatizer(list_of_tokens):
    
    lemmatized_tokens_list = []
    
    for i in list_of_tokens: 
        token = WordNetLemmatizer().lemmatize(i)
        lemmatized_tokens_list.append(token)
        
    return lemmatized_tokens_list


def the_untokenizer(token_list):
        '''
        Returns all the tokenized words in the list to one string. 
        Used after the pre processing, such as removing stopwords, and lemmatizing. 
        '''
        return " ".join(token_list)
    
def cleaning_our_texts(text):
    
    cleaned_texts = []
    num_texts = len(text)
    
    for i in text.index:
        that_string = text[i]
        tokenized_list = tokenizer(that_string)
        removed_stopwords = remove_stopwords(tokenized_list)
        stemmed_words = stemmer(removed_stopwords)
        lemmatized_words = lemmatizer(stemmed_words)
        back_to_string = the_untokenizer(lemmatized_words)
        
        cleaned_texts.append(back_to_string)
    
    return cleaned_texts

In [12]:
%%time
X['plot_clean'] = cleaning_our_texts(X['plot_long'])
X = X.drop(['plot_long'],axis=1)

Wall time: 1min 6s


In [13]:
X

,f_release_year,f_release_month,f_runtime,f_word_count_long,f_imdb_rating,f_num_imdb_votes,f_num_user_reviews,f_num_critic_reviews,plot_clean
0,1920,2,67,170,8.1,52815,222.0,156.0,franci young man recal memori horribl experi f...
1,1920,6,91,33,7.2,6136,49.0,45.0,In 16th-centuri pragu rabbi creat golem - gian...
2,1920,9,145,155,7.4,4800,49.0,28.0,the callou rich portray lennox think pleasur a...
3,1920,9,82,135,7.0,4564,71.0,65.0,base robert loui stevenson stori : doctor henr...
4,1920,1,79,179,6.3,2139,20.0,13.0,southern negro sylvia landri visit cousin alma...
...,...,...,...,...,...,...,...,...,...
30040,2019,12,100,147,1.4,1347,19.0,0.0,the stori friendship famili witch make easi wa...
30041,2019,11,91,91,5.5,1066,60.0,19.0,blue stori tragic tale friendship timmi marco ...
30042,2019,4,90,38,4.2,1010,43.0,1.0,A good man ( tip `` t.i '' harri ) return home...
30043,2019,3,123,16,6.7,1057,5.0,36.0,A griev widow move countri chanc encount rekin...


In [14]:
%%time
# Now it's time for the NLP preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=20) #, ngram_range=(1,3))

# Fit the vectorizer to the training data
tfidf.fit(X['plot_clean'])

X_tfidf = tfidf.transform(X['plot_clean'])

joblib.dump(tfidf, 'models/my_best_tfidf.pkl')

Wall time: 2.79 s


['models/my_best_tfidf.pkl']

In [15]:
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=X.index, columns=tfidf.get_feature_names()) 
X_tfidf_df

,00,000,10,100,1000,11,12,13,13th,14,15,16,16th,17,17th,18,1800,1890,18th,19,1900,1920,1930,1936,1939,1940,1941,1942,1943,1944,1945,1946,1947,1950,1953,1955,1956,1957,1959,1960,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1998,1999,19th,20,200,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,20th,21,21st,22,23,24,25,250,26,27,28,29,2nd,30,300,32,35,36,40,400,42,45,48,4th,50,500,60,70,80,90,aaron,abandon,abba,abbi,abduct,abel,abid,abil,abl,aboard,abort,abound,about,abraham,abroad,abruptli,absenc,absent,absolut,absorb,absurd,abus,academ,academi,accept,access,accid,accident,acclaim,accommod,accompani,accomplic,accomplish,accord,account,accus,accustom,ace,achiev,acid,acknowledg,acp,acquaint,acquir,across,act,action,activ,activist,actor,actress,actual,ad,adam,adapt,add,addict,addit,address,aditya,adjust,administr,admir,admiss,admit,adolesc,adolf,adopt,ador,adrian,adrift,adult,adulteri,adulthood,advanc,advantag,adventur,adversari,advertis,advic,advis,advisor,advoc,affair,affect,afflict,affluent,afford,afghanistan,afraid,africa,african,aft,after,afterlif,aftermath,afternoon,afterward,ag,against,age,agenc,agenda,agent,aggress,agn,ago,agre,agreement,ahead,ahm,ai,aid,ail,aim,aimless,air,aircraft,airlin,airplan,airport,ajay,aka,...,visit,visitor,visual,vital,vivaci,vivian,vivid,vladimir,vocal,voic,void,volatil,volcan,volcano,volunt,von,voodoo,vote,vow,voyag,vs,vulgar,vulner,wa,wacki,wade,wage,wagner,wagon,wait,waiter,waitress,wake,wale,walk,walker,wall,wallac,wallet,walt,walter,wander,wane,wang,wannab,want,war,ward,warden,warehous,warfar,warlord,warm,warn,warner,warren,warrior,warsaw,wartim,wash,washington,wast,wasteland,watch,water,watson,watt,wave,way,wayn,wayward,we,weak,weaken,wealth,wealthi,weapon,wear,weari,weather,weav,web,websit,wed,weed,week,weekend,weekli,wei,weight,weird,welcom,welfar,well,welsh,wendi,went,werewolf,west,western,wet,whale,what,whatev,wheel,wheelchair,when,whenev,where,wherea,whereabout,wherev,whether,whi,which,while,whilst,whip,whirlwind,whisper,whit,white,who,whoever,whole,whore,whose,wick,wide,widow,wield,wif,wife,wil,wild,wilder,wildli,wildlif,will,willi,william,wilson,win,wind,window,wine,wing,winner,winston,winter,wipe,wire,wis,wisconsin,wisdom,wise,wish,wit,witch,witchcraft,with,withdraw,withdrawn,within,without,witti,wive,wizard,wo,wolf,wolv,woman,won,wonder,wong,woo,wood,wooden,woodi,word,work,workahol,worker,working,world,worldli,worldwid,worm,worri,wors,worsen,worship,worst,worth,worthi,would,wound,wrap,wrath,wreak,wreck,wreckag,wrench,wrestl,wrestler,wright,write,writer,written,wrong,wrongli,wrote,wu,ww2,wwi,wwii,wyatt,wyom,xavier,yacht,yakuza,yang,yanke,yard,ye,year,yearn,yellow,yet,york,yorker,you,young,younger,youngest,youngster,your,youth,youtub,yu,yugoslavia,yuppi,zach,zack,zealand,zero,zoe,zombi,zone,zoo
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.068322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [18]:
X_df_full = X.join(X_tfidf_df, on=X_tfidf_df.index)
X_df_full.shape

X = X_df_full.drop(['plot_clean'], axis=1).copy()
#df = df.merge(y, on=df.index)
#df = df.set_index(['key_0'])
#del df.index.name
#df
X

,f_release_year,f_release_month,f_runtime,f_word_count_long,f_imdb_rating,f_num_imdb_votes,f_num_user_reviews,f_num_critic_reviews,00,000,10,100,1000,11,12,13,13th,14,15,16,16th,17,17th,18,1800,1890,18th,19,1900,1920,1930,1936,1939,1940,1941,1942,1943,1944,1945,1946,1947,1950,1953,1955,1956,1957,1959,1960,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1998,1999,19th,20,200,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,20th,21,21st,22,23,24,25,250,26,27,28,29,2nd,30,300,32,35,36,40,400,42,45,48,4th,50,500,60,70,80,90,aaron,abandon,abba,abbi,abduct,abel,abid,abil,abl,aboard,abort,abound,about,abraham,abroad,abruptli,absenc,absent,absolut,absorb,absurd,abus,academ,academi,accept,access,accid,accident,acclaim,accommod,accompani,accomplic,accomplish,accord,account,accus,accustom,ace,achiev,acid,acknowledg,acp,acquaint,acquir,across,act,action,activ,activist,actor,actress,actual,ad,adam,adapt,add,addict,addit,address,aditya,adjust,administr,admir,admiss,admit,adolesc,adolf,adopt,ador,adrian,adrift,adult,adulteri,adulthood,advanc,advantag,adventur,adversari,advertis,advic,advis,advisor,advoc,affair,affect,afflict,affluent,afford,afghanistan,afraid,africa,african,aft,after,afterlif,aftermath,afternoon,afterward,ag,against,age,agenc,agenda,agent,aggress,agn,ago,agre,agreement,ahead,ahm,ai,aid,ail,aim,...,visit,visitor,visual,vital,vivaci,vivian,vivid,vladimir,vocal,voic,void,volatil,volcan,volcano,volunt,von,voodoo,vote,vow,voyag,vs,vulgar,vulner,wa,wacki,wade,wage,wagner,wagon,wait,waiter,waitress,wake,wale,walk,walker,wall,wallac,wallet,walt,walter,wander,wane,wang,wannab,want,war,ward,warden,warehous,warfar,warlord,warm,warn,warner,warren,warrior,warsaw,wartim,wash,washington,wast,wasteland,watch,water,watson,watt,wave,way,wayn,wayward,we,weak,weaken,wealth,wealthi,weapon,wear,weari,weather,weav,web,websit,wed,weed,week,weekend,weekli,wei,weight,weird,welcom,welfar,well,welsh,wendi,went,werewolf,west,western,wet,whale,what,whatev,wheel,wheelchair,when,whenev,where,wherea,whereabout,wherev,whether,whi,which,while,whilst,whip,whirlwind,whisper,whit,white,who,whoever,whole,whore,whose,wick,wide,widow,wield,wif,wife,wil,wild,wilder,wildli,wildlif,will,willi,william,wilson,win,wind,window,wine,wing,winner,winston,winter,wipe,wire,wis,wisconsin,wisdom,wise,wish,wit,witch,witchcraft,with,withdraw,withdrawn,within,without,witti,wive,wizard,wo,wolf,wolv,woman,won,wonder,wong,woo,wood,wooden,woodi,word,work,workahol,worker,working,world,worldli,worldwid,worm,worri,wors,worsen,worship,worst,worth,worthi,would,wound,wrap,wrath,wreak,wreck,wreckag,wrench,wrestl,wrestler,wright,write,writer,written,wrong,wrongli,wrote,wu,ww2,wwi,wwii,wyatt,wyom,xavier,yacht,yakuza,yang,yanke,yard,ye,year,yearn,yellow,yet,york,yorker,you,young,younger,youngest,youngster,your,youth,youtub,yu,yugoslavia,yuppi,zach,zack,zealand,zero,zoe,zombi,zone,zoo
0,1920,2,67,170,8.1,52815,222.0,156.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.068322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,

In [19]:
%%time
from sklearn.preprocessing import StandardScaler

my_standard_scaler = StandardScaler().fit(X)
X_scaled = my_standard_scaler.transform(X)

joblib.dump(my_standard_scaler, 'models/my_best_scaler.pkl')

Wall time: 5.74 s


['models/my_best_scaler.pkl']

In [20]:
%%time
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

my_best_model = OneVsRestClassifier(LogisticRegression(random_state=123, solver='lbfgs', max_iter=3000, C=0.01, n_jobs=-1), n_jobs=-1).fit(X_scaled, y)

Wall time: 2min


In [21]:
joblib.dump(my_best_model, 'models/my_best_model.pkl')

['models/my_best_model.pkl']